In [1]:
from pathlib import Path

from pcd_dataset.kitti_dataset import KittiDataset

dataset_path = 'dataset/'
sequence = '00'
sam_labels_path = Path.cwd().joinpath('pipeline', 'vfm-labels', 'sam', sequence)

kitti = KittiDataset(dataset_path, sequence, sam_labels_path)

In [2]:
from mapping.simple_mapping import SimpleMapping

sm = SimpleMapping(kitti)
coo_matrix, pcds = sm.get_combined_labeled_point_clouds('cam2', 20, 23)

INFO:simple_mapping.SimpleMapping:Image 20 processed
INFO:simple_mapping.SimpleMapping:Image 21 processed
INFO:simple_mapping.SimpleMapping:Image 22 processed


In [3]:
from mapping.sparse_matrix_utils import construct_coo_matrix_for_multiple_views

coo_result = construct_coo_matrix_for_multiple_views(coo_matrix, 20)

In [4]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse_output=True)

In [5]:
enc.fit((coo_result.T).toarray())

OneHotEncoder(handle_unknown='ignore')

In [6]:
enc.categories_

[array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10., 11., 12., 13.,
        14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
        27., 28., 29., 30., 31., 32., 33., 34., 35., 37., 38., 39., 40.,
        42., 43., 44., 45., 46., 48., 49., 50., 51., 53., 54., 56., 57.,
        58., 59., 62., 63., 64., 67., 68., 69., 71., 73., 74., 75., 76.,
        77., 78., 79., 80.]),
 array([ 0.,  2.,  3.,  4.,  5.,  6.,  7., 10., 11., 12., 13., 14., 15.,
        16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27., 28.,
        29., 30., 31., 32., 33., 34., 35., 36., 37., 39., 40., 41., 42.,
        43., 44., 45., 47., 49., 50., 51., 52., 54., 55., 56., 58., 59.,
        60., 62., 64., 65., 67., 69., 71., 72., 73., 74.]),
 array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
        13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
        26., 27., 28., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39.,
        40., 41., 42., 43., 44., 4

In [7]:
enc_coo_result = enc.transform(coo_result.T.toarray()).toarray()

In [8]:
inverse_enc_coo_result = enc.inverse_transform(enc_coo_result)

k = 0
n = 0
for itr in range(len(inverse_enc_coo_result)):
    if (inverse_enc_coo_result[itr] == [0.0, 0.0, 0.0]).all():
        k = k + 1
    if (inverse_enc_coo_result[itr] == [0.0, 0.0, 0.0]).any():
        n = n + 1
        
print("Number of points with non-zero all instances: {}".format(len(inverse_enc_coo_result) - k))
print("Number of points with non-zero any instances: {}".format(len(inverse_enc_coo_result) - n))

Number of points with non-zero all instances: 92411
Number of points with non-zero any instances: 1629


In [9]:
summary_pcd = pcds[0] + pcds[1] + pcds[2]

In [10]:
len(inverse_enc_coo_result)

832606

In [11]:
len(summary_pcd.points)

832606

In [12]:
inverse_enc_coo_result = enc.inverse_transform(enc_coo_result)
points = []
map_old_new_point_ind = {}

k = 0
for itr in range(len(inverse_enc_coo_result)):
    if (inverse_enc_coo_result[itr] == [0.0, 0.0, 0.0]).all():
        k = k + 1
    else:
        points.append(summary_pcd.points[itr])
        dict1 = {itr:(len(points) - 1)}
        map_old_new_point_ind.update(dict1)

In [13]:
map_old_new_point_ind

{0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 5,
 20: 6,
 21: 7,
 22: 8,
 23: 9,
 24: 10,
 25: 11,
 26: 12,
 27: 13,
 28: 14,
 32: 15,
 33: 16,
 34: 17,
 35: 18,
 36: 19,
 37: 20,
 38: 21,
 39: 22,
 40: 23,
 41: 24,
 42: 25,
 43: 26,
 44: 27,
 45: 28,
 46: 29,
 47: 30,
 48: 31,
 49: 32,
 50: 33,
 51: 34,
 52: 35,
 53: 36,
 54: 37,
 55: 38,
 56: 39,
 57: 40,
 58: 41,
 59: 42,
 60: 43,
 61: 44,
 62: 45,
 63: 46,
 64: 47,
 65: 48,
 66: 49,
 67: 50,
 68: 51,
 69: 52,
 70: 53,
 71: 54,
 72: 55,
 73: 56,
 74: 57,
 75: 58,
 76: 59,
 77: 60,
 78: 61,
 79: 62,
 80: 63,
 81: 64,
 82: 65,
 83: 66,
 84: 67,
 85: 68,
 86: 69,
 87: 70,
 88: 71,
 89: 72,
 90: 73,
 91: 74,
 92: 75,
 93: 76,
 94: 77,
 95: 78,
 96: 79,
 97: 80,
 98: 81,
 99: 82,
 100: 83,
 101: 84,
 102: 85,
 103: 86,
 104: 87,
 105: 88,
 106: 89,
 107: 90,
 108: 91,
 109: 92,
 110: 93,
 111: 94,
 112: 95,
 113: 96,
 114: 97,
 115: 98,
 1153: 99,
 1154: 100,
 1155: 101,
 1156: 102,
 1157: 103,
 1158: 104,
 1159: 105,
 1160: 106,
 1161: 107,
 1

In [14]:
map_old_new_point_ind[78]

61

In [15]:
enc_coo_non_zero_instances = []
for itr in range(len(enc_coo_result)):
    if itr in map_old_new_point_ind.keys():
        enc_coo_non_zero_instances.append(enc_coo_result[itr])

In [16]:
inverse_enc_coo_non_zero_instances = enc.inverse_transform(enc_coo_non_zero_instances)
print(inverse_enc_coo_non_zero_instances[61])
print(inverse_enc_coo_result[78])

[48. 41. 50.]
[48. 41. 50.]


In [17]:
import open3d as o3d

pcd_baby = o3d.geometry.PointCloud()
pcd_baby.points = o3d.utility.Vector3dVector(points)

In [18]:
downpcd = pcd_baby.voxel_down_sample(voxel_size=0.05)
# o3d.visualization.draw_geometries([downpcd])

In [19]:
downpcd.points[50000]

array([-2.79080391,  1.66868822,  7.49760649])

In [20]:
for it in range(len(downpcd.points)):
    if (downpcd.points[it] == [-2.7908039129797504, 1.668688224242099, 7.497606486148587]).all():
        print("ooo")

ooo


In [21]:
from sklearn.metrics.pairwise import pairwise_distances
distances = pairwise_distances(enc_coo_non_zero_instances[:20000])

In [22]:
distances.shape

(20000, 20000)

In [ ]:
from awesome_normalized_cut import normalized_cut

clusters = normalized_cut(distances, pcd_baby.points[20000], 1.1)

1
(20000, 20000)
2
3
4
5
6
7


In [54]:
clusters

[array([272, 692, 878]),
 array([ 61, 563, 736, 891, 903]),
 array([127, 252, 764, 941]),
 array([ 36, 185, 423, 668, 893]),
 array([ 75, 110, 140, 335, 678]),
 array([433, 583, 687]),
 array([477]),
 array([323, 956]),
 array([ 60, 224, 503, 683, 823]),
 array([ 94, 234, 499]),
 array([ 22, 177, 375]),
 array([277, 539, 721]),
 array([373, 554]),
 array([397]),
 array([457]),
 array([ 44, 439, 633, 693, 901]),
 array([321, 528, 826]),
 array([871]),
 array([ 27,  49, 237, 567, 857]),
 array([429, 611]),
 array([182, 379, 415]),
 array([197, 362, 628, 756, 869]),
 array([384]),
 array([128]),
 array([124, 691, 763]),
 array([419, 607, 629]),
 array([ 82, 133, 656]),
 array([441, 625]),
 array([204, 349, 380, 483]),
 array([939]),
 array([ 21, 157, 209, 403, 815]),
 array([14]),
 array([100, 636]),
 array([ 99, 132, 690, 970]),
 array([390, 626, 972]),
 array([299, 463, 806]),
 array([313, 920]),
 array([351]),
 array([597, 684, 750, 935]),
 array([428, 556]),
 array([307, 367, 377, 879

In [55]:
inverse_enc_coo_non_zero_instances[61]

array([48., 41., 50.])

In [56]:
inverse_enc_coo_non_zero_instances[563]

array([ 0., 13., 30.])

In [43]:
clusters[0][9999]

array([9.29407103, 0.40789549, 0.18582554])

In [12]:
# 1) выбросить ненужные точки. создать новое облако и мапу старое - новое
# 2) voxel_down_sample применить к маленькому облаку
# применить к нему граф кат, раскрасить по кластерам ИСХОДНОЕ маленькое облако с новыми инстансами
# 3) kd деревья почитать чтобы найти соседей